In [6]:
import pandas as pd
import requests
import json
from pprint import pprint
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import psycopg2
from config import database_url

In [7]:
# File to load
movie= "movies_metadata.csv"
file2 = "nextBechdel_castGender.csv"
file3 = "MoviesOnStreamingPlatforms_updated.csv"

In [8]:
# Read csv file
file2 = "nextBechdel_castGender.csv"
movies_data= pd.read_csv(movie)
movies_data.head()

C:\Users\becky\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
# Remove columns 
movies_data["production_countries"]
movies_df= movies_data.drop(columns=["adult","belongs_to_collection", "homepage", "imdb_id", "original_title", 
                                  "runtime", "spoken_languages", "tagline", "poster_path", "video","genres", "original_language",
                                    "overview", "popularity","production_companies","vote_average","vote_count"])
movies_df

,budget,id,production_countries,release_date,revenue,status,title
0,30000000,862,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,Released,Toy Story
1,65000000,8844,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,Released,Jumanji
2,0,15602,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,Released,Grumpier Old Men
3,16000000,31357,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,Released,Waiting to Exhale
4,0,11862,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,Released,Father of the Bride Part II
...,...,...,...,...,...,...,...
45461,0,439050,"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,Released,Subdue
45462,0,111109,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,Released,Century of Birthing
45463,0,67758,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,Released,Betrayal
45464,0,227506,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,Released,Satan Triumphant


In [10]:
movies_US_df= movies_df[movies_df["status"] == "Released"]
movies_US_df

,budget,id,production_countries,release_date,revenue,status,title
0,30000000,862,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,Released,Toy Story
1,65000000,8844,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,Released,Jumanji
2,0,15602,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,Released,Grumpier Old Men
3,16000000,31357,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,Released,Waiting to Exhale
4,0,11862,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,Released,Father of the Bride Part II
...,...,...,...,...,...,...,...
45461,0,439050,"[{'iso_3166_1': 'IR', 'name': 'Iran'}]",NaN,0.0,Released,Subdue
45462,0,111109,"[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",2011-11-17,0.0,Released,Century of Birthing
45463,0,67758,"[{'iso_3166_1': 'US', 'name': 'United States o...",2003-08-01,0.0,Released,Betrayal
45464,0,227506,"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",1917-10-21,0.0,Released,Satan Triumphant


In [11]:
movies_US_df['status'].unique()

array(['Released'], dtype=object)

In [12]:
# Movies produced only in the US
movies_US_df= movies_US_df[movies_US_df["production_countries"] == "[{'iso_3166_1': 'US', 'name': 'United States of America'}]"]
movies_US_df

,budget,id,production_countries,release_date,revenue,status,title
0,30000000,862,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,Released,Toy Story
1,65000000,8844,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,Released,Jumanji
2,0,15602,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,Released,Grumpier Old Men
3,16000000,31357,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,Released,Waiting to Exhale
4,0,11862,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,Released,Father of the Bride Part II
...,...,...,...,...,...,...,...
45454,0,420346,"[{'iso_3166_1': 'US', 'name': 'United States o...",2015-01-11,0.0,Released,The Morning After
45456,0,84419,"[{'iso_3166_1': 'US', 'name': 'United States o...",1946-03-29,0.0,Released,House of Horrors
45458,0,289923,"[{'iso_3166_1': 'US', 'name': 'United States o...",2000-10-03,0.0,Released,The Burkittsville 7
45459,0,222848,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-01-01,0.0,Released,Caged Heat 3000


In [13]:
#drop production country column
movies_filtered_df = movies_US_df.drop(columns=["production_countries","budget","revenue"])
movies_filtered_df

,id,release_date,status,title
0,862,1995-10-30,Released,Toy Story
1,8844,1995-12-15,Released,Jumanji
2,15602,1995-12-22,Released,Grumpier Old Men
3,31357,1995-12-22,Released,Waiting to Exhale
4,11862,1995-02-10,Released,Father of the Bride Part II
...,...,...,...,...
45454,420346,2015-01-11,Released,The Morning After
45456,84419,1946-03-29,Released,House of Horrors
45458,289923,2000-10-03,Released,The Burkittsville 7
45459,222848,1995-01-01,Released,Caged Heat 3000


In [14]:
streaming = pd.read_csv(file3)
streaming.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [15]:
streaming = streaming.drop(columns=['Unnamed: 0', 'ID','Age', 'IMDb', 'Rotten Tomatoes','Type', 'Directors',
       'Genres', 'Country', 'Language', 'Runtime'])
streaming_clean = streaming.rename(columns = {'Title': 'title', 
                                              'Year': 'year', 
                                              'Netflix': 'netflix', 
                                              'Hulu':'hulu',
                                              'Prime Video':'prime_video',
                                             'Disney+': 'disney+'})
streaming_clean

,title,year,netflix,hulu,prime_video,disney+
0,Inception,2010,1,0,0,0
1,The Matrix,1999,1,0,0,0
2,Avengers: Infinity War,2018,1,0,0,0
3,Back to the Future,1985,1,0,0,0
4,"The Good, the Bad and the Ugly",1966,1,0,1,0
...,...,...,...,...,...,...
16739,The Ghosts of Buxley Hall,1980,0,0,0,1
16740,The Poof Point,2001,0,0,0,1
16741,Sharks of Lost Island,2013,0,0,0,1
16742,Man Among Cheetahs,2017,0,0,0,1


In [16]:
movies_clean = pd.merge(movies_filtered_df,streaming_clean,on='title')
movies_filtered_df['status'].unique()

array(['Released'], dtype=object)

In [17]:
movies_filtered_df

,id,release_date,status,title
0,862,1995-10-30,Released,Toy Story
1,8844,1995-12-15,Released,Jumanji
2,15602,1995-12-22,Released,Grumpier Old Men
3,31357,1995-12-22,Released,Waiting to Exhale
4,11862,1995-02-10,Released,Father of the Bride Part II
...,...,...,...,...
45454,420346,2015-01-11,Released,The Morning After
45456,84419,1946-03-29,Released,House of Horrors
45458,289923,2000-10-03,Released,The Burkittsville 7
45459,222848,1995-01-01,Released,Caged Heat 3000


In [26]:
movies_clean

,id,release_date,status,title,year,netflix,hulu,prime_video,disney+
0,862,1995-10-30,Released,Toy Story,1995,0,0,0,1
1,31357,1995-12-22,Released,Waiting to Exhale,1995,0,1,0,0
2,45325,1995-12-22,Released,Tom and Huck,1995,0,0,0,1
3,33689,1995-11-17,Released,It Takes Two,1995,1,0,1,0
4,110538,1988-07-13,Released,It Takes Two,1995,1,0,1,0
...,...,...,...,...,...,...,...,...,...
3620,417320,2017-07-21,Released,Descendants 2,2017,0,0,0,1
3621,435473,2017-02-03,Released,American Violence,2017,0,0,1,0
3622,426272,2017-05-05,Released,Take Me,2017,1,0,0,0
3623,461297,2017-08-01,Released,Cop and a Half: New Recruit,2017,1,0,0,0


In [19]:
#API url to get initial list of movies in dataset
bechdel_url = 'http://bechdeltest.com/api/v1/getAllMovies'

response = requests.get(bechdel_url)
raw_data = response.json()
pprint(raw_data)

[{'id': 9602,
  'imdbid': '3155794',
  'rating': 0,
  'title': 'Passage de Venus',
  'year': 1874},
 {'id': 9804,
  'imdbid': '14495706',
  'rating': 0,
  'title': 'La Rosace Magique',
  'year': 1877},
 {'id': 9806,
  'imdbid': '12592084',
  'rating': 0,
  'title': 'Le singe musicien',
  'year': 1878},
 {'id': 9603,
  'imdbid': '2221420',
  'rating': 0,
  'title': 'Sallie Gardner at a Gallop',
  'year': 1878},
 {'id': 9816,
  'imdbid': '7816420',
  'rating': 0,
  'title': 'Athlete Swinging a Pick',
  'year': 1881},
 {'id': 9831,
  'imdbid': '5459794',
  'rating': 0,
  'title': 'Buffalo Running',
  'year': 1883},
 {'id': 9832,
  'imdbid': '8588366',
  'rating': 0,
  'title': 'L&#39;homme machine',
  'year': 1885},
 {'id': 9836,
  'imdbid': '8133192',
  'rating': 0,
  'title': 'Cockatoo Flying',
  'year': 1887},
 {'id': 9843,
  'imdbid': '7796044',
  'rating': 0,
  'title': 'Woman Opening Umbrella',
  'year': 1887},
 {'id': 9846,
  'imdbid': '7917370',
  'rating': 0,
  'title': 'Woman Ju

  'rating': 0,
  'title': 'A Trip to Mars',
  'year': 1910},
 {'id': 1464,
  'imdbid': '0232821',
  'rating': 3,
  'title': 'Tilly the Tomboy Visits the Poor',
  'year': 1910},
 {'id': 4983,
  'imdbid': '0486978',
  'rating': 0,
  'title': 'Mobilier fidele',
  'year': 1910},
 {'id': 2003,
  'imdbid': '0002101',
  'rating': 2,
  'title': 'Cleopatra',
  'year': 1912},
 {'id': 5357,
  'imdbid': '0277865',
  'rating': 0,
  'title': 'Max and the Donkey',
  'year': 1912},
 {'id': 2005,
  'imdbid': '0002281',
  'rating': 0,
  'title': 'Invaders, The',
  'year': 1912},
 {'id': 5364,
  'imdbid': '0003512',
  'rating': 0,
  'title': 'Voyage of the Bourrichon Family, The',
  'year': 1913},
 {'id': 5915,
  'imdbid': '0003424',
  'rating': 0,
  'title': 'Suspense',
  'year': 1913},
 {'id': 4457,
  'imdbid': '0003973',
  'rating': 2,
  'title': 'A Florida Enchantment',
  'year': 1914},
 {'id': 1258,
  'imdbid': '0004972',
  'rating': 2,
  'title': 'Birth of a Nation, The',
  'year': 1915},
 {'id': 2

  'year': 1928},
 {'id': 1314,
  'imdbid': '0019254',
  'rating': 0,
  'title': 'Passion of Joan of Arc, The',
  'year': 1928},
 {'id': 5984,
  'imdbid': '0018839',
  'rating': 2,
  'title': 'Docks of New York, The',
  'year': 1928},
 {'id': 1972,
  'imdbid': '0019304',
  'rating': 0,
  'title': 'Racket, The',
  'year': 1928},
 {'id': 1979,
  'imdbid': '0019290',
  'rating': 1,
  'title': 'Power of the Press, The',
  'year': 1928},
 {'id': 1968,
  'imdbid': '0018773',
  'rating': 0,
  'title': 'Circus, The',
  'year': 1928},
 {'id': 3897,
  'imdbid': '0019760',
  'rating': 0,
  'title': 'Man with a Movie Camera',
  'year': 1929},
 {'id': 1953,
  'imdbid': '0019777',
  'rating': 2,
  'title': 'Cocoanuts, The',
  'year': 1929},
 {'id': 5979,
  'imdbid': '0019702',
  'rating': 2,
  'title': 'Blackmail',
  'year': 1929},
 {'id': 8464,
  'imdbid': '0019729',
  'rating': 3,
  'title': 'Broadway Melody',
  'year': 1929},
 {'id': 6046,
  'imdbid': '0020286',
  'rating': 0,
  'title': 'Throw of

  'title': 'Footlight Parade',
  'year': 1933},
 {'id': 5029,
  'imdbid': '0023932',
  'rating': 3,
  'title': 'Day of Reckoning',
  'year': 1933},
 {'id': 1270,
  'imdbid': '0024991',
  'rating': 1,
  'title': 'Cleopatra',
  'year': 1934},
 {'id': 621,
  'imdbid': '0025316',
  'rating': 3,
  'title': 'It Happened One Night',
  'year': 1934},
 {'id': 8653,
  'imdbid': '0025543',
  'rating': 0,
  'title': 'Mystery Liner',
  'year': 1934},
 {'id': 4443,
  'imdbid': '0025822',
  'rating': 3,
  'title': 'Spitfire',
  'year': 1934},
 {'id': 7304,
  'imdbid': '0025452',
  'rating': 3,
  'title': 'Man Who Knew Too Much, The',
  'year': 1934},
 {'id': 1795,
  'imdbid': '0024844',
  'rating': 0,
  'title': 'L&#39;atalante',
  'year': 1934},
 {'id': 4702,
  'imdbid': '0025465',
  'rating': 3,
  'title': 'Maniac',
  'year': 1934},
 {'id': 5523,
  'imdbid': '0025424',
  'rating': 2,
  'title': 'Lost in the Stratosphere',
  'year': 1934},
 {'id': 1329,
  'imdbid': '0025617',
  'rating': 1,
  'title

  'rating': 3,
  'title': 'Four Daughters',
  'year': 1938},
 {'id': 1378,
  'imdbid': '0030755',
  'rating': 2,
  'title': 'Sisters, The',
  'year': 1938},
 {'id': 593,
  'imdbid': '0031971',
  'rating': 3,
  'title': 'Stagecoach',
  'year': 1939},
 {'id': 1385,
  'imdbid': '0031981',
  'rating': 2,
  'title': 'Story of Alexander Graham Bell, The',
  'year': 1939},
 {'id': 9038,
  'imdbid': '0031593',
  'rating': 3,
  'title': 'Love Affair',
  'year': 1939},
 {'id': 2622,
  'imdbid': '0031680',
  'rating': 1,
  'title': 'Mr. Wong in Chinatown',
  'year': 1939},
 {'id': 1245,
  'imdbid': '0031385',
  'rating': 2,
  'title': 'Goodbye, Mr. Chips',
  'year': 1939},
 {'id': 1259,
  'imdbid': '0031725',
  'rating': 3,
  'title': 'Ninotchka',
  'year': 1939},
 {'id': 2851,
  'imdbid': '0031742',
  'rating': 1,
  'title': 'Of Mice and Men',
  'year': 1939},
 {'id': 1370,
  'imdbid': '0031210',
  'rating': 3,
  'title': 'Dark Victory',
  'year': 1939},
 {'id': 2372,
  'imdbid': '0031060',
  'r

...

  'imdbid': '4633694',
  'rating': 1,
  'title': 'Spider-Man : Into the Spider-Verse',
  'year': 2018},
 {'id': 8588,
  'imdbid': '7125860',
  'rating': 3,
  'title': 'If Beale Street Could Talk',
  'year': 2018},
 {'id': 8178,
  'imdbid': '4154756',
  'rating': 3,
  'title': 'Avengers: Infinity War',
  'year': 2018},
 {'id': 8656,
  'imdbid': '6205872',
  'rating': 3,
  'title': 'Assassination Nation',
  'year': 2018},
 {'id': 8319,
  'imdbid': '7156222',
  'rating': 0,
  'title': 'Au poste!',
  'year': 2018},
 {'id': 8351,
  'imdbid': '7242142',
  'rating': 1,
  'title': 'Blindspotting',
  'year': 2018},
 {'id': 9127, 'imdbid': '4357394', 'rating': 3, 'title': 'Tau', 'year': 2018},
 {'id': 8754, 'imdbid': '7098658', 'rating': 3, 'title': 'Raazi', 'year': 2018},
 {'id': 8383,
  'imdbid': '3458254',
  'rating': 1,
  'title': 'Patient Zero',
  'year': 2018},
 {'id': 9235,
  'imdbid': '5980638',
  'rating': 3,
  'title': 'Transcendents, The',
  'year': 2018},
 {'id': 8497,
  'imdbid': '6

 {'id': 9066,
  'imdbid': '5936578',
  'rating': 3,
  'title': 'Saints Rest',
  'year': 2018},
 {'id': 8324,
  'imdbid': '5688932',
  'rating': 1,
  'title': 'Sorry to Bother You',
  'year': 2018},
 {'id': 9132,
  'imdbid': '6889032',
  'rating': 3,
  'title': 'Cake General, The',
  'year': 2018},
 {'id': 8792,
  'imdbid': '7134690',
  'rating': 3,
  'title': 'I Like Me',
  'year': 2018},
 {'id': 8423,
  'imdbid': '0365545',
  'rating': 3,
  'title': 'Nappily Ever Aftet',
  'year': 2018},
 {'id': 8567,
  'imdbid': '5437928',
  'rating': 3,
  'title': 'Colette',
  'year': 2018},
 {'id': 8137,
  'imdbid': '5104604',
  'rating': 3,
  'title': 'Isle of Dogs',
  'year': 2018},
 {'id': 8290,
  'imdbid': '3606756',
  'rating': 3,
  'title': 'Incredibles 2',
  'year': 2018},
 {'id': 8734,
  'imdbid': '7689958',
  'rating': 3,
  'title': 'Jane Fonda in Five Acts',
  'year': 2018},
 {'id': 8395,
  'imdbid': '7485508',
  'rating': 3,
  'title': 'Carmen y Lola',
  'year': 2018},
 {'id': 8431, 'imd

  'imdbid': '8323104',
  'rating': 3,
  'title': 'Nancy Drew and the Hidden Staircase',
  'year': 2019},
 {'id': 9159,
  'imdbid': '6803046',
  'rating': 3,
  'title': 'Vast of Night, The',
  'year': 2019},
 {'id': 8786, 'imdbid': '7456310', 'rating': 3, 'title': 'Anna', 'year': 2019},
 {'id': 8862,
  'imdbid': '4682804',
  'rating': 3,
  'title': 'Lucy in the Sky',
  'year': 2019},
 {'id': 8895,
  'imdbid': '7349950',
  'rating': 3,
  'title': 'It Chapter 2',
  'year': 2019},
 {'id': 9067,
  'imdbid': '3038708',
  'rating': 3,
  'title': 'Iron Sky: The Coming Race',
  'year': 2019},
 {'id': 8728,
  'imdbid': '5941692',
  'rating': 3,
  'title': 'Miss Bala',
  'year': 2019},
 {'id': 8760,
  'imdbid': '7374948',
  'rating': 3,
  'title': 'Always Be My Maybe',
  'year': 2019},
 {'id': 8869,
  'imdbid': '1950186',
  'rating': 0,
  'title': 'Ford v Ferrari',
  'year': 2019},
 {'id': 9278,
  'imdbid': '8758086',
  'rating': 3,
  'title': 'Mercy Black',
  'year': 2019},
 {'id': 8903,
  'imdb

  'year': 2019},
 {'id': 8781,
  'imdbid': '6320628',
  'rating': 1,
  'title': 'Spider-Man: Far from Home',
  'year': 2019},
 {'id': 8815,
  'imdbid': '1478839',
  'rating': 2,
  'title': 'Art of Racing in the Rain, The',
  'year': 2019},
 {'id': 9222,
  'imdbid': '9217514',
  'rating': 3,
  'title': 'Une Paire de Jumelles',
  'year': 2019},
 {'id': 9452,
  'imdbid': '9902160',
  'rating': 3,
  'title': 'Herself',
  'year': 2020},
 {'id': 9658,
  'imdbid': '10441958',
  'rating': 3,
  'title': 'How to Be a Good Wife',
  'year': 2020},
 {'id': 9786,
  'imdbid': '12198524',
  'rating': 0,
  'title': 'Retreat, The',
  'year': 2020},
 {'id': 9031,
  'imdbid': '11895476',
  'rating': 3,
  'title': 'Veil, The',
  'year': 2020},
 {'id': 9491,
  'imdbid': '9893250',
  'rating': 3,
  'title': 'I Care a Lot',
  'year': 2020},
 {'id': 9135,
  'imdbid': '12261880',
  'rating': 3,
  'title': 'Methodists, The',
  'year': 2020},
 {'id': 9499,
  'imdbid': '6317656',
  'rating': 3,
  'title': 'Penguin

  'title': 'last days of american crime, The',
  'year': 2020},
 {'id': 9200,
  'imdbid': '6048922',
  'rating': 0,
  'title': 'Greyhound',
  'year': 2020},
 {'id': 9003,
  'imdbid': '7510346',
  'rating': 3,
  'title': 'Turning, The',
  'year': 2020},
 {'id': 9496,
  'imdbid': '6475714',
  'rating': 3,
  'title': 'Monster Hunter',
  'year': 2020},
 {'id': 9172,
  'imdbid': '9308382',
  'rating': 3,
  'title': 'High Note, The',
  'year': 2020},
 {'id': 9669,
  'imdbid': '13079086',
  'rating': 1,
  'title': 'From Stress to Happiness',
  'year': 2020},
 {'id': 9284,
  'imdbid': '4682266',
  'rating': 3,
  'title': 'New Mutants, The',
  'year': 2020},
 {'id': 9470,
  'imdbid': '9820556',
  'rating': 1,
  'title': 'Breach',
  'year': 2020},
 {'id': 9179,
  'imdbid': '8244784',
  'rating': 3,
  'title': 'Hunt, The',
  'year': 2020},
 {'id': 9255,
  'imdbid': '1086064',
  'rating': 3,
  'title': 'Bill &amp; Ted Face the Music',
  'year': 2020},
 {'id': 9397,
  'imdbid': '9086228',
  'rating

In [20]:
len(raw_data)

9169

In [21]:
#create dataframe from initial API request
bechdel_all = pd.DataFrame(raw_data,columns = ['id','imdbid','rating','title','year'])

bechdel_all
#rating bechdel test:  
    # 0 = no two women
    # 1 = no talking
    # 2 = talking about a man
    # 3 = passes bechdel test


,id,imdbid,rating,title,year
0,9602,3155794,0,Passage de Venus,1874
1,9804,14495706,0,La Rosace Magique,1877
2,9806,12592084,0,Le singe musicien,1878
3,9603,2221420,0,Sallie Gardner at a Gallop,1878
4,9816,7816420,0,Athlete Swinging a Pick,1881
...,...,...,...,...,...
9164,9830,0499097,1,Without Remorse,2021
9165,9500,9286908,2,High Ground,2021
9166,9599,8521718,3,"United States vs. Billie Holiday, The",2021
9167,9635,10121392,3,Thunder Force,2021


In [22]:
bechdel_final = bechdel_all[bechdel_all['year']>=2000]
bechdel_final

,id,imdbid,rating,title,year
3514,5743,0244870,3,Spiral,2000
3515,5073,0263215,3,Picture Claire,2000
3516,625,0181984,1,Boiler Room,2000
3517,4863,0235712,3,Ring 0: Birthday (Ringu 0: B&acirc;sudei),2000
3518,5711,0235327,0,Dark Days,2000
...,...,...,...,...,...
9164,9830,0499097,1,Without Remorse,2021
9165,9500,9286908,2,High Ground,2021
9166,9599,8521718,3,"United States vs. Billie Holiday, The",2021
9167,9635,10121392,3,Thunder Force,2021


In [23]:
# Add to ETL_project_db
engine =create_engine(database_url)
connection = engine.connect()
bechdel_final.to_sql('bechdel', con=engine, if_exists='replace')
engine.execute("SELECT * from bechdel").fetchall()



[(3514, 5743, '0244870', 3, 'Spiral', 2000),
 (3515, 5073, '0263215', 3, 'Picture Claire', 2000),
 (3516, 625, '0181984', 1, 'Boiler Room', 2000),
 (3517, 4863, '0235712', 3, 'Ring 0: Birthday (Ringu 0: B&acirc;sudei)', 2000),
 (3518, 5711, '0235327', 0, 'Dark Days', 2000),
 (3519, 1664, '0206036', 3, 'If These Walls Could Talk 2', 2000),
 (3520, 319, '0134847', 3, 'Pitch Black', 2000),
 (3521, 1206, '0214641', 0, 'Dragonheart: A New Beginning', 2000),
 (3522, 90, '0183523', 1, 'Mission to Mars', 2000),
 (3523, 875, '0250440', 3, 'Is It Fall Yet', 2000),
 (3524, 4703, '0202677', 2, 'Way of the Gun, The', 2000),
 (3525, 3950, '0163978', 1, 'Beach, The', 2000),
 (3526, 8769, '0248254', 1, 'Old School', 2000),
 (3527, 6442, '0330500', 3, 'Ju-on', 2000),
 (3528, 4600, '0247380', 3, 'Gleaners and I, The', 2000),
 (3529, 882, '0198021', 3, 'Where the Heart Is', 2000),
 (3530, 131, '0190138', 3, 'Whole Nine Yards, The', 2000),
 (3531, 1277, '0266308', 3, 'Battle Royale', 2000),
 (3532, 6380, 

In [30]:
engine =create_engine(database_url)
connection = engine.connect()
try:
    movies_clean.to_sql('movies', con=engine, if_exists='replace')
except:
    print("Error")
engine.execute("SELECT * from movies").fetchall()


[(0, '862', '1995-10-30', 'Released', 'Toy Story', 1995, 0, 0, 0, 1),
 (1, '31357', '1995-12-22', 'Released', 'Waiting to Exhale', 1995, 0, 1, 0, 0),
 (2, '45325', '1995-12-22', 'Released', 'Tom and Huck', 1995, 0, 0, 0, 1),
 (3, '33689', '1995-11-17', 'Released', 'It Takes Two', 1995, 1, 0, 1, 0),
 (4, '110538', '1988-07-13', 'Released', 'It Takes Two', 1995, 1, 0, 1, 0),
 (5, '35196', '1995-12-29', 'Released', 'Restoration', 2011, 0, 0, 1, 0),
 (6, '9312', '1995-08-18', 'Released', 'Mortal Kombat', 1995, 1, 0, 0, 0),
 (7, '10530', '1995-06-14', 'Released', 'Pocahontas', 1995, 0, 0, 0, 1),
 (8, '9089', '1995-11-02', 'Released', 'Home for the Holidays', 1995, 0, 0, 1, 0),
 (9, '30933', '1974-07-13', 'Released', 'Home for the Holidays', 1995, 0, 0, 1, 0),
 (10, '17182', '1996-01-12', 'Released', 'Eye for an Eye', 2019, 1, 0, 1, 0),
 (11, '9536', '1996-01-12', 'Released', 'Bio-Dome', 1996, 0, 0, 1, 0),
 (12, '28387', '1995-10-06', 'Released', 'Kicking and Screaming', 1995, 1, 0, 0, 0),
 